[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Bigeco/AI_Segmentation_Competition_2023/blob/master/notebooks/check_data_to_image.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from IPython.display import Image, display

from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps

In [ ]:
# Pytorch에서 gpu를 사용하는 방법.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

In [ ]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)["image"]
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"]

        return image, mask

In [ ]:
# Data Loader
transform = A.Compose([A.Resize(224, 224), A.Normalize(), ToTensorV2()])

dataset = SatelliteDataset(csv_file="/content/drive/MyDrive/sw_ai/train.csv", transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

---

# 1. 변수 input_img_paths, target_img_mask_rle_decode 정의

- `input_img_paths` | input_path 그대로 적용하면 안열리므로 바꿔줌.
- `target_img_mask_rle_decode` | mask_rel 데이터에서 이미지로 만들기 위해 binary 수로 바꿔야 한다. 그래서 decode을 적용하고 list에 삽입

In [ ]:
dataset.data

In [ ]:
train_dir = "/content/drive/MyDrive/sw_ai/train.csv"
test_dir = "/content/drive/MyDrive/sw_ai/test.csv"
img_size = (1024, 1024)
# num_classes = 2
# batch_size =

In [ ]:
input_img_paths = dataset.data["img_path"]
input_img_paths = ["/content/drive/MyDrive/sw_ai/" + path[2:] for path in input_img_paths]
target_img_mask_rle_decode = [
    rle_decode(dataset.data["mask_rle"][i], img_size) for i in range(10)
]  # RLE 인코딩 된 예측 이진 마스크

In [ ]:
rle_decode(dataset.data["mask_rle"][0], img_size)

In [ ]:
# check image_size
# im = cv2.imread(input_img_paths[0], 0)
# print(im.shape)

---

# 2. 이미지 출력
- input image 출력 시 `IPython.display` 의 `Image`, `display` 사용
- target image 출력 시 `PIL`의 `Image`, 그리고 `struct` 사용  (위 Image와 이름이 동일해서 as Imagee로 씀)

In [ ]:
# input image 출력
display(Image(filename = input_img_paths[0]))

In [ ]:
from PIL import Image as Imagee
import struct

In [ ]:
size = 1024, 1024
arr = sum(target_img_mask_rle_decode[0].tolist(), [])
data = struct.pack('B'*len(arr), *[pixel*255 for pixel in arr])
img = Imagee.frombuffer('L', size, data)
img.show()